In [ ]:
!pip install catboost

     |████████████████████████████████| 65.9MB 58kB/s 


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

In [ ]:
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_ = pd.read_csv('Train.csv')
test_ = pd.read_csv('Test.csv')
submission_ = pd.read_csv('SampleSubmission.csv')

In [ ]:
print(train_.shape)
train_.head()

(29132, 29)


,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,1/6/2013,M,U,1991,748L,QZYX,90QI,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,1/8/2019,M,M,1990,1X1H,BP09,56SI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,1/8/2019,M,M,1990,748L,NO3L,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [ ]:
print(test_.shape)
test_.head()

(10000, 29)


,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,F86J5PC,1/12/2018,M,M,1984,94KC,DZRV,90QI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H6141K3,1/10/2019,M,M,1996,1X1H,J9SY,90QI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,RBAYUXZ,1/1/2020,F,W,1968,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,KCBILBQ,1/2/2019,M,M,1989,94KC,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,LSEC1ZJ,1/2/2020,F,M,1982,UAOD,0KID,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
print(submission_.shape)
submission_.head()

(210000, 2)


,ID X PCODE,Label
0,F86J5PC X P5DA,0
1,F86J5PC X RIBP,0
2,F86J5PC X 8NN1,0
3,F86J5PC X 7POT,0
4,F86J5PC X 66FJ,0


In [ ]:
from sklearn.model_selection import KFold

def get_train_test_names(train_, test_, submission_):
  kf = KFold(n_splits=5, shuffle=False)
  for r, (train_index, test_index) in enumerate(kf.split(train_)):
    test = train_.iloc[test_index]

    X_test = []
    X_test_columns = test.columns
    for v in test.values:
      info = v[:8]
      binary = v[8:]
      index = [k for k, i in enumerate(binary) if i == 1]
      for i in index:
        for k in range(len(binary)):
          if k == i:
            binary_transformed = list(copy.copy(binary))
            binary_transformed[i] = 0
            X_test.append(list(info) + binary_transformed)

    X_test = pd.DataFrame(X_test)
    X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
          'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
          '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
          'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']

    X_test['ID'] = [str(r)+'_'+str(i) for i in range(X_test.shape[0])]

    yield train_.iloc[train_index], X_test, submission_, '1_fold' + str(r) + '.csv'
  yield train_, test_, submission_, '1_main.csv'

### Get folds

In [ ]:
for train, test, submission, name in get_train_test_names(train_, test_, submission_):

  np_data = []
  train_columns = train.columns
  for v in tqdm(train.values):
    info = v[:8]
    binary = v[8:]
    index_n = [k for k, i in enumerate(binary) if i == 1]
    for i in index_n:
      for k in range(len(binary)):
        if (k not in index_n) or (k == i):
          binary_0 = list(copy.copy(binary))
          binary_0[i] = 0
          if k == i:
            np_data.append(list(info) + binary_0 + [train_columns[8+k]] + [1])
          else:
            np_data.append(list(info) + binary_0 + [train_columns[8+k]] + [0])

  df_data = pd.DataFrame(np_data)
  df_data.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'target']

  np_data_test = []
  answ_test = []
  test_columns = test.columns
  for v in tqdm(test.values):
    info = v[:8]
    binary = v[8:]
    index_n = [k for k, i in enumerate(binary) if i == 1]
    for k in range(len(binary)):
      if k not in index_n:
        np_data_test.append(list(info) + list(binary) + [test_columns[8+k]])

  df_data_test = pd.DataFrame(np_data_test)
  df_data_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred']

  df_data['date1'] = df_data['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
  df_data['date2'] = df_data['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
  df_data['date3'] = df_data['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
  df_data.drop('join_date', axis=1, inplace=True)

  df_data_test['date1'] = df_data_test['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
  df_data_test['date2'] = df_data_test['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
  df_data_test['date3'] = df_data_test['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
  df_data_test.drop('join_date', axis=1, inplace=True)

  df_data['date_diff'] = df_data['date3'] - df_data['birth_year']
  df_data_test['date_diff'] = df_data_test['date3'] - df_data_test['birth_year']

  from catboost import CatBoostClassifier
  cat_features = ['sex', 'marital_status', 'branch_code',
        'occupation_code', 'occupation_category_code', 'product_pred']

  model = CatBoostClassifier()
  model.fit(df_data.drop(['ID', 'target'], axis=1), df_data['target'], cat_features)

  preds_proba = model.predict_proba(df_data_test.drop(['ID',], axis=1))
  df_answer = df_data_test[['ID', 'product_pred']]
  df_answer['target'] = preds_proba[:,1]
  df_answer['ID X PCODE'] = df_answer['ID'] + ' X ' + df_answer['product_pred']
  df_answer.drop(['ID', 'product_pred'], axis=1, inplace=True)
  df_answer.rename(columns={'target':'Label'}, inplace=True)
  df_answer = submission[submission['ID X PCODE'].isin(list(set(list(submission['ID X PCODE'])) - set(list(df_answer['ID X PCODE']))))].append(df_answer)
  df_answer.reset_index(drop=True, inplace=True)
  df_answer.to_csv(name, index=False)

100%|██████████| 13262/13262 [00:01<00:00, 9056.46it/s]


Learning rate set to 0.199962
0:	learn: 0.2919641	total: 1.48s	remaining: 24m 33s
1:	learn: 0.1621258	total: 2.53s	remaining: 21m 4s
2:	learn: 0.1134291	total: 3.52s	remaining: 19m 29s
3:	learn: 0.0878900	total: 4.92s	remaining: 20m 26s
4:	learn: 0.0764236	total: 6.11s	remaining: 20m 16s
5:	learn: 0.0709430	total: 7.16s	remaining: 19m 46s
6:	learn: 0.0678081	total: 8.4s	remaining: 19m 51s
7:	learn: 0.0597583	total: 9.74s	remaining: 20m 7s
8:	learn: 0.0577007	total: 11s	remaining: 20m 5s
9:	learn: 0.0562532	total: 12.1s	remaining: 19m 54s
10:	learn: 0.0550062	total: 13.1s	remaining: 19m 40s
11:	learn: 0.0537892	total: 14.3s	remaining: 19m 40s
12:	learn: 0.0528309	total: 15.7s	remaining: 19m 50s
13:	learn: 0.0523708	total: 17s	remaining: 19m 57s
14:	learn: 0.0499179	total: 18.3s	remaining: 20m 2s
15:	learn: 0.0494869	total: 19.4s	remaining: 19m 51s
16:	learn: 0.0487618	total: 20.7s	remaining: 19m 57s
17:	learn: 0.0483397	total: 22.1s	remaining: 20m 4s
18:	learn: 0.0479066	total: 23.5s	re

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

Learning rate set to 0.200026
0:	learn: 0.2925427	total: 1.1s	remaining: 18m 16s
1:	learn: 0.1623938	total: 2.27s	remaining: 18m 54s
2:	learn: 0.1135185	total: 3.24s	remaining: 17m 57s
3:	learn: 0.0879687	total: 4.38s	remaining: 18m 11s
4:	learn: 0.0765315	total: 5.61s	remaining: 18m 35s
5:	learn: 0.0709079	total: 6.76s	remaining: 18m 40s
6:	learn: 0.0653503	total: 7.8s	remaining: 18m 27s
7:	learn: 0.0621624	total: 8.85s	remaining: 18m 18s
8:	learn: 0.0573443	total: 10.2s	remaining: 18m 45s
9:	learn: 0.0558431	total: 11.4s	remaining: 18m 51s
10:	learn: 0.0535605	total: 12.6s	remaining: 18m 50s
11:	learn: 0.0526961	total: 13.6s	remaining: 18m 41s
12:	learn: 0.0520021	total: 14.7s	remaining: 18m 37s
13:	learn: 0.0515101	total: 16s	remaining: 18m 44s
14:	learn: 0.0505128	total: 17.3s	remaining: 18m 56s
15:	learn: 0.0497593	total: 18.8s	remaining: 19m 16s
16:	learn: 0.0480684	total: 20.3s	remaining: 19m 34s
17:	learn: 0.0476362	total: 21.6s	remaining: 19m 35s
18:	learn: 0.0471248	total: 22

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

...

Learning rate set to 0.199904
0:	learn: 0.2904477	total: 1.32s	remaining: 21m 57s
1:	learn: 0.1625516	total: 2.46s	remaining: 20m 25s
2:	learn: 0.1138985	total: 3.4s	remaining: 18m 51s
3:	learn: 0.0882042	total: 4.73s	remaining: 19m 38s
4:	learn: 0.0767381	total: 5.77s	remaining: 19m 8s
5:	learn: 0.0705262	total: 7.26s	remaining: 20m 2s
6:	learn: 0.0670883	total: 8.54s	remaining: 20m 12s
7:	learn: 0.0601760	total: 9.76s	remaining: 20m 10s
8:	learn: 0.0583543	total: 10.9s	remaining: 19m 57s
9:	learn: 0.0567677	total: 11.9s	remaining: 19m 42s
10:	learn: 0.0559261	total: 13.1s	remaining: 19m 39s
11:	learn: 0.0549624	total: 14.3s	remaining: 19m 34s
12:	learn: 0.0537304	total: 15.9s	remaining: 20m 6s
13:	learn: 0.0504681	total: 17.4s	remaining: 20m 27s
14:	learn: 0.0493272	total: 18.9s	remaining: 20m 39s
15:	learn: 0.0487345	total: 20.2s	remaining: 20m 43s
16:	learn: 0.0483977	total: 21.4s	remaining: 20m 37s
17:	learn: 0.0480432	total: 22.7s	remaining: 20m 39s
18:	learn: 0.0476029	total: 24

100%|██████████| 13307/13307 [00:01<00:00, 12612.17it/s]


Learning rate set to 0.199902
0:	learn: 0.2893962	total: 1.26s	remaining: 20m 55s
1:	learn: 0.1600847	total: 2.35s	remaining: 19m 35s
2:	learn: 0.1132501	total: 3.28s	remaining: 18m 11s
3:	learn: 0.0878837	total: 4.62s	remaining: 19m 11s
4:	learn: 0.0767205	total: 5.93s	remaining: 19m 39s
5:	learn: 0.0694637	total: 6.89s	remaining: 19m 1s
6:	learn: 0.0662237	total: 8.11s	remaining: 19m 11s
7:	learn: 0.0634916	total: 9.34s	remaining: 19m 18s
8:	learn: 0.0586687	total: 10.6s	remaining: 19m 22s
9:	learn: 0.0576035	total: 11.7s	remaining: 19m 16s
10:	learn: 0.0536876	total: 12.9s	remaining: 19m 21s
11:	learn: 0.0527230	total: 13.8s	remaining: 18m 59s
12:	learn: 0.0520899	total: 15.1s	remaining: 19m 8s
13:	learn: 0.0513475	total: 16.6s	remaining: 19m 30s
14:	learn: 0.0497886	total: 18s	remaining: 19m 43s
15:	learn: 0.0492713	total: 19.2s	remaining: 19m 41s
16:	learn: 0.0484717	total: 20.7s	remaining: 19m 56s
17:	learn: 0.0478901	total: 22s	remaining: 20m 2s
18:	learn: 0.0474617	total: 23.4s

100%|██████████| 10000/10000 [00:00<00:00, 11896.06it/s]


Learning rate set to 0.219943
0:	learn: 0.2716990	total: 1.65s	remaining: 27m 26s
1:	learn: 0.1474916	total: 3.43s	remaining: 28m 31s
2:	learn: 0.1034731	total: 5.19s	remaining: 28m 45s
3:	learn: 0.0851117	total: 6.49s	remaining: 26m 55s
4:	learn: 0.0740160	total: 8.06s	remaining: 26m 43s
5:	learn: 0.0671631	total: 9.3s	remaining: 25m 40s
6:	learn: 0.0628554	total: 11s	remaining: 25m 59s
7:	learn: 0.0597708	total: 12.1s	remaining: 25m
8:	learn: 0.0581049	total: 13.3s	remaining: 24m 26s
9:	learn: 0.0524992	total: 15.3s	remaining: 25m 16s
10:	learn: 0.0515805	total: 17.1s	remaining: 25m 40s
11:	learn: 0.0503702	total: 19.1s	remaining: 26m 13s
12:	learn: 0.0495171	total: 21.1s	remaining: 26m 40s
13:	learn: 0.0489759	total: 22.7s	remaining: 26m 37s
14:	learn: 0.0483282	total: 24.5s	remaining: 26m 46s
15:	learn: 0.0478308	total: 26.1s	remaining: 26m 48s
16:	learn: 0.0474760	total: 27.9s	remaining: 26m 54s
17:	learn: 0.0467492	total: 29.9s	remaining: 27m 12s
18:	learn: 0.0458775	total: 31.6s

OK